In [120]:
import pandas as pd
import numpy as np
import datetime as dt
from math import floor

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

df = pd.read_csv('/Users/yoannabriel/Documents/GitHub/Rush5PDG4/DataOut.csv', sep=",")
df_copy = df.copy()
df.head()

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1
1,1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0
2,2,4141,1965,Graduation,relationship,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0
3,3,6182,1984,Graduation,relationship,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0
4,4,5324,1981,PhD,relationship,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0


In [121]:
#feature engineering 
df = df.set_index('ID')

In [122]:
df['Age'] = int(pd.datetime.now().year) - df['Year_Birth']
df_copy['Age'] = int(pd.datetime.now().year) - df_copy['Year_Birth']

In [123]:
print("Columns with string datatype are:")
for col in df.columns:
    if df[col].dtypes == object:
        print(col)

Columns with string datatype are:
Education
Marital_Status
Dt_Customer


In [124]:
df_Edu = pd.DataFrame(df['Education'].value_counts()).reset_index()
df_Edu.columns = ['Education', 'Count']

In [125]:
df['Education'] = np.where(df['Education'] == '2n Cycle', 'Master', df['Education'])

In [126]:
df_Edu = pd.DataFrame(df['Education'].value_counts()).reset_index()
df_Edu.columns = ['Education', 'Count']

fig = px.bar(df_Edu,
            x='Education',
            y='Count',
            color='Education')
fig.update_layout(width=800, height=400, title='Education ')
fig.show()

In [127]:
# df['h# df['hasKid'] = np.where((df['Kidhome'] + df['Teenhome'] > 0), 1, 0)
df['Kids'] = df['Kidhome'] + df['Teenhome']

df['Kids'].describe()

count    2240.000000
mean        0.950446
std         0.751803
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         3.000000
Name: Kids, dtype: float64

In [128]:
df_Mar = pd.DataFrame(df['Marital_Status'].value_counts()).reset_index()
df_Mar.columns = ['Marital_Status', 'Count']
df_Mar

,Marital_Status,Count
0,relationship,1444
1,Single,796


In [129]:
df_Edu = pd.DataFrame(df['Marital_Status'].value_counts()).reset_index()
df_Edu.columns = ['Marital_Status', 'Count']

fig = px.bar(df_Edu,
            x='Marital_Status',
            y='Count',
            color='Marital_Status')
fig.update_layout(width=800, height=400, title='Education ')
fig.show()

In [130]:
df.head()

,Unnamed: 0,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Kids
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,0,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,67,0
2174,1,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,70,2
4141,2,1965,Graduation,relationship,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,59,0
6182,3,1984,Graduation,relationship,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,40,1
5324,4,1981,PhD,relationship,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,43,1


In [131]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], utc=False)

In [132]:
last_enrollment = pd.to_datetime(df['Dt_Customer'].max(), utc=False)
df['Days_Enrolled'] = (last_enrollment-df['Dt_Customer']).dt.days

In [133]:
df.drop(columns = ['Year_Birth', 'Kidhome', 'Teenhome'], inplace=True)
df.head()

,Unnamed: 0,Education,Marital_Status,Income,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Kids,Days_Enrolled
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,0,Graduation,Single,58138.0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,67,0,663
2174,1,Graduation,Single,46344.0,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,70,2,113
4141,2,Graduation,relationship,71613.0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,59,0,312
6182,3,Graduation,relationship,26646.0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,40,1,139
5324,4,PhD,relationship,58293.0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,43,1,161


In [134]:
df['Total_Spend'] = df['MntWines']+df['MntFruits']+df['MntMeatProducts']+df['MntFishProducts']+df['MntSweetProducts']+df['MntGoldProds']

df.drop(columns=['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds'], inplace=True)

print(f"Customer with most recent non transaction streak is {df['Recency'].min()} days while customer with longest non transaction streak is {df['Recency'].max()} days")


Customer with most recent non transaction streak is 0 days while customer with longest non transaction streak is 99 days


In [135]:
df[df['Recency'] <= 7].shape

(184, 22)

In [136]:
df[(df['Recency']>7) & (df['Recency'] <= 14 )].shape

(171, 22)

In [137]:
df[(df['Recency']>14) & (df['Recency'] <= 21 )].shape


(143, 22)

In [138]:
df['Recency_'] = df['Recency']/14
df['Recency_'] = df['Recency_'].apply(lambda x: floor(x))

In [139]:
df_Recency_ = pd.DataFrame(df['Recency_'].value_counts()).reset_index()
df_Recency_.columns = ['Recency', 'Count']

fig = px.bar(df_Recency_,
            x='Recency',
            y='Count',
            color='Recency')
fig.update_layout(width=800, height=400, title='Recency (x14 days) ')
fig.show()


In [140]:
df['Recency_'] = df['Recency_'].apply(lambda x : str(x))
df.drop(columns = ['Recency'], inplace=True)
df.head()


,Unnamed: 0,Education,Marital_Status,Income,Dt_Customer,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Kids,Days_Enrolled,Total_Spend,Recency_
ID,,,,,,,,,,,,,,,,,,,,,,
5524,0,Graduation,Single,58138.0,2012-09-04,3,8,10,4,7,0,0,0,0,0,0,1,67,0,663,1617,4
2174,1,Graduation,Single,46344.0,2014-03-08,2,1,1,2,5,0,0,0,0,0,0,0,70,2,113,27,2
4141,2,Graduation,relationship,71613.0,2013-08-21,1,8,2,10,4,0,0,0,0,0,0,0,59,0,312,776,1
6182,3,Graduation,relationship,26646.0,2014-02-10,2,2,0,4,6,0,0,0,0,0,0,0,40,1,139,53,1
5324,4,PhD,relationship,58293.0,2014-01-19,5,5,3,6,5,0,0,0,0,0,0,0,43,1,161,422,6


In [141]:
df['CouponsAccepted'] = df['AcceptedCmp1']+df['AcceptedCmp2']+df['AcceptedCmp3']+df['AcceptedCmp4']+df['AcceptedCmp5']+df['Response']
df.drop(columns=['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response', 'NumWebVisitsMonth'], inplace=True)

In [142]:
df.rename(columns={'NumStorePurchases':'Store_Purchases'}, inplace=True)
df['Non_Store_Purchases'] = df['NumWebPurchases']+df['NumCatalogPurchases']

In [144]:
df['Total_Purchases'] = df['Store_Purchases']+df['Non_Store_Purchases']
df['Store_Purchases_Pct'] = df['Store_Purchases']/df['Total_Purchases']
df['Non_Store_Purchases_Pct'] = 1-df['Store_Purchases_Pct']
df['Customer_Type'] = np.where(df['Store_Purchases_Pct'] >= 0.5, "Offline", "Online")
df.head()


,Unnamed: 0,Education,Marital_Status,Income,Dt_Customer,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,Store_Purchases,Complain,Age,Kids,Days_Enrolled,Total_Spend,Recency_,CouponsAccepted,Non_Store_Purchases,Total_Purchases,Store_Purchases_Pct,Non_Store_Purchases_Pct,Customer_Type
ID,,,,,,,,,,,,,,,,,,,,,
5524,0,Graduation,Single,58138.0,2012-09-04,3,8,10,4,0,67,0,663,1617,4,1,18,22,0.181818,0.818182,Online
2174,1,Graduation,Single,46344.0,2014-03-08,2,1,1,2,0,70,2,113,27,2,0,2,4,0.500000,0.500000,Offline
4141,2,Graduation,relationship,71613.0,2013-08-21,1,8,2,10,0,59,0,312,776,1,0,10,20,0.500000,0.500000,Offline
6182,3,Graduation,relationship,26646.0,2014-02-10,2,2,0,4,0,40,1,139,53,1,0,2,6,0.666667,0.333333,Offline
5324,4,PhD,relationship,58293.0,2014-01-19,5,5,3,6,0,43,1,161,422,6,0,8,14,0.428571,0.571429,Online


In [145]:
df.drop(columns=['Store_Purchases_Pct', 'Non_Store_Purchases_Pct', 'Non_Store_Purchases', 'NumWebPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'Store_Purchases'], inplace=True)


In [146]:
print(f"The youngest customer is {df['Age'].min()} years old and oldest customer is {df['Age'].max()} years old")

The youngest customer is 28 years old and oldest customer is 131 years old


In [ ]:
trace0 = go.Box(
    y=df['Income'],
    name='Income'
)

trace1 = go.Box(
    y=df['Age'],
    name='Age'
)

trace2 = go.Box(
    y=df['NumDealsPurchases'],
    name='Deals Purchases'
)

trace3 = go.Box(
    y=df['Total_Spend'],
    name='Total Spend'
)

trace4 = go.Box(
    y=df['CouponsAccepted'],
    name='Coupons Accepted'
)

trace5 = go.Box(
    y=df['Total_Purchases'],
    name='Total Purchases'
)



fig = make_subplots(rows=2, cols=3)

fig.append_trace(trace0, row = 1, col = 1)
fig.append_trace(trace1, row = 1, col = 2)
fig.append_trace(trace2, row = 1, col = 3)
fig.append_trace(trace3, row = 2, col = 1)
fig.append_trace(trace4, row = 2, col = 2)
fig.append_trace(trace5, row = 2, col = 3)

fig.update_layout(width=800, height=400, title='Box Plot to check for outliers')

fig.show()